In [54]:
import gzip
import json
import os
import numpy as np
import pickle
from operator import itemgetter

import gzip
import json
import os
import numpy as np
import pickle
from operator import itemgetter

from gensim.models import KeyedVectors
from nltk.corpus import stopwords

from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from keras import Model, Input
from keras.layers import Embedding, Dropout, Bidirectional, LSTM, K, Lambda, Dense, RepeatVector, TimeDistributed, \
    merge, Activation, Flatten, Permute, Reshape
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from keras.regularizers import l2
from os import listdir
from os.path import isfile, join

import keras
from keras.regularizers import l2
from keras.callbacks import *
from keras.models import *
from keras.optimizers import *
from keras.utils.np_utils import to_categorical
from keras.layers.core import *
from keras.layers import Input, Embedding, LSTM, Dense, merge, TimeDistributed
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K

from sklearn.model_selection import train_test_split
from keras import metrics
from sklearn.metrics import confusion_matrix as cm

import pandas as pd
from pandas import DataFrame
from pandas.io.json import json_normalize  
pd.set_option('display.max_colwidth', 50)

from sklearn.metrics import confusion_matrix
import random

# from DataLoader import DataLoader
from table import WikiTable

import sys
sys.setrecursionlimit(100000) 

In [4]:
'''
    Compute P/R/F1 from the confusion matrix.
'''
def evaluation_metrics_report(mat, labels_, method_, epochs=10):
    num_classes = len(mat)
    
    scores = dict()
    avg_p = []
    avg_r = []
    avg_f1 = []
    for i in range(0, num_classes):
        p = mat[i,i] / float(sum(mat[:,i]))
        r = mat[i,i] / float(sum(mat[i,:]))
        f1 = 2 * (p * r) / (p + r)
        
        scores[i] = (p, r, f1)
        avg_p.append(p)
        avg_r.append(r)
        avg_f1.append(f1)
    
    outstr = 'Evaluation results for ' + method_ + ' Epochs: ' + str(epochs) + '\n'
    for key in scores:
        label = labels_[key]
        val_1 = scores[key][0]
        val_2 = scores[key][1]
        val_3 = scores[key][2]
        
        outstr += ('%s\tP=%.3f\tR=%.3f\tF1=%.3f\n' % (label, val_1, val_2, val_3))
    avg_p_score = sum(avg_p) / len(avg_p)
    avg_r_score = sum(avg_r) / len(avg_r)
    avg_f1_score = sum(avg_f1) / len(avg_f1)
    outstr += 'AVG\tAvg-P=%.3f\tAvg-R=%.3f\tAvg-F1=%.3f\n' % (avg_p_score, avg_r_score, avg_f1_score)
    return outstr

# LSTM column-by-column with attention model.

In [5]:
def get_H_n(X):
    ans = X[:, -1, :]  # get last element from time dim
    return ans


def get_Y(X, xmaxlen):
    return X[:, :xmaxlen, :]  # get first xmaxlen elem from time dim


def get_R(X):
    Y, alpha = X[0], X[1]
    ans = K.batch_dot(Y, alpha)
    return ans


'''
    LSTM baseline where the columns are represented by their title, and LCA category.
'''


def build_lstm_baseline_w2v(w2v_size, w2v_dim, word2vec_matrix, lstm_units=100, col_length=10):
    k = 2 * lstm_units
    N = col_length

    # the first layer is the embeddings for the column names
    main_input = Input(shape=(N,), dtype='int32', name='main_input')
    e = Embedding(w2v_size, w2v_dim, weights=[word2vec_matrix], input_length=N, trainable=False)(main_input)

    # merge the two embedding layers
    drop_out = Dropout(0.3, name='dropout')(e)
    # pass the input through a BiLSTM model through the previous layer
    lstm = LSTM(lstm_units, return_sequences=True)(drop_out)
    # add another drop-out layer
    drop_out = Dropout(0.2)(lstm)
    flatten = Flatten()(drop_out)
    out = Dense(3, activation='softmax')(flatten)
    
    model = Model(input=main_input, output=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
    return model

'''
    LSTM baseline where the columns are represented by their title, and LCA category.
'''


def build_lstm_baseline_w2v_lca(w2v_size, w2v_dim, word2vec_matrix, n2v_size, n2v_dim, node2vec_matrix, lstm_units=100, col_length=10):
    k = 2 * lstm_units
    N = col_length

    # the first layer is the embeddings for the column names
    main_input = Input(shape=(N,), dtype='int32', name='main_input')
    e = Embedding(w2v_size, w2v_dim, weights=[word2vec_matrix], input_length=N, trainable=False)(main_input)

    
    # add the second input layer which corresponds to the category embeddings
    subject_input = Input(shape=(N,), dtype='int32', name='subject_input')
    e2 = Embedding(n2v_size, n2v_dim, weights=[node2vec_matrix], input_length=N, trainable=False)(subject_input)

    # merge the two embedding layers
    e4 = merge([e, e2], mode='sum')

    # merge the two embedding layers
    drop_out = Dropout(0.3, name='dropout')(e4)
    # pass the input through a BiLSTM model through the previous layer
    lstm = LSTM(lstm_units, return_sequences=True)(drop_out)
    # add another drop-out layer
    drop_out = Dropout(0.2)(lstm)
    flatten = Flatten()(drop_out)
    out = Dense(3, activation='softmax')(flatten)
    
    model = Model(input=[main_input, subject_input], output=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
    return model


'''
    LSTM baseline where the columns are represented by their title, and LCA category.
'''


def build_lstm_baseline_w2v_lca_val(w2v_size, w2v_dim, word2vec_matrix, n2v_size, n2v_dim, node2vec_matrix, lstm_units=100, col_length=10):
    k = 2 * lstm_units
    N = col_length

    # the first layer is the embeddings for the column names
    main_input = Input(shape=(N,), dtype='int32', name='main_input')
    e = Embedding(w2v_size, w2v_dim, weights=[word2vec_matrix], input_length=N, trainable=False)(main_input)

    
    # add the second input layer which corresponds to the category embeddings
    subject_input = Input(shape=(N,), dtype='int32', name='subject_input')
    e2 = Embedding(n2v_size, n2v_dim, weights=[node2vec_matrix], input_length=N, trainable=False)(subject_input)
    
        # add the second input layer which corresponds to the category embeddings
    value_input = Input(shape=(N,), dtype='int32', name='value_input')
    e3 = Embedding(n2v_size, n2v_dim, weights=[node2vec_matrix], input_length=N, trainable=False)(value_input)


    # merge the two embedding layers
    e4 = merge([e, e2, e3], mode='sum')

    # merge the two embedding layers
    drop_out = Dropout(0.3, name='dropout')(e4)
    # pass the input through a BiLSTM model through the previous layer
    lstm = LSTM(lstm_units, return_sequences=True)(drop_out)
    # add another drop-out layer
    drop_out = Dropout(0.2)(lstm)
    flatten = Flatten()(drop_out)
    out = Dense(3, activation='softmax')(flatten)
    
    model = Model(input=[main_input, subject_input, value_input], output=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
    return model

'''
    BiLSTM baseline where the columns are represented by their title, and LCA category.
'''


def build_bilstm_baseline_w2v_lca(w2v_size, w2v_dim, word2vec_matrix, n2v_size, n2v_dim, node2vec_matrix, lstm_units=100, col_length=10):
    k = 2 * lstm_units
    N = col_length

    # the first layer is the embeddings for the column names
    main_input = Input(shape=(N,), dtype='int32', name='main_input')
    e = Embedding(w2v_size, w2v_dim, weights=[word2vec_matrix], input_length=N, trainable=False)(main_input)

    
    # add the second input layer which corresponds to the category embeddings
    subject_input = Input(shape=(N,), dtype='int32', name='subject_input')
    e2 = Embedding(n2v_size, n2v_dim, weights=[node2vec_matrix], input_length=N, trainable=False)(subject_input)

    # merge the two embedding layers
    e4 = merge([e, e2], mode='sum')

    # merge the two embedding layers
    drop_out = Dropout(0.3, name='dropout')(e4)
    # pass the input through a BiLSTM model through the previous layer
    bilstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(drop_out)
    # add another drop-out layer
    drop_out = Dropout(0.2)(bilstm)
    flatten = Flatten()(drop_out)
    out = Dense(3, activation='softmax')(flatten)
    
    model = Model(input=[main_input, subject_input], output=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
    return model



'''
    BiLSTM baseline where the columns are represented by their title.
'''


def build_bilstm_baseline_w2v(w2v_size, w2v_dim, word2vec_matrix, lstm_units=100, col_length=10):
    k = 2 * lstm_units
    N = col_length

    # the first layer is the embeddings for the column names
    main_input = Input(shape=(N,), dtype='int32', name='main_input')
    e = Embedding(w2v_size, w2v_dim, weights=[word2vec_matrix], input_length=N, trainable=False)(main_input)

    # merge the two embedding layers
    drop_out = Dropout(0.3, name='dropout')(e)
    # pass the input through a BiLSTM model through the previous layer
    bilstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(drop_out)

    # add another drop-out layer
    drop_out = Dropout(0.2)(bilstm)
    flatten = Flatten()(drop_out)
    out = Dense(3, activation='softmax')(flatten)
    output = out

    model = Model(input=main_input, output=output)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
    return model


'''
    BiLSTM baseline where the columns are represented by their title, LCA category, and column values.
'''


def build_bilstm_baseline_w2v_lca_val(w2v_size, w2v_dim, word2vec_matrix, n2v_size, n2v_dim, node2vec_matrix, lstm_units=100, col_length=10):
    k = 2 * lstm_units
    N = col_length

    # the first layer is the embeddings for the column names
    main_input = Input(shape=(N,), dtype='int32', name='main_input')
    e = Embedding(w2v_size, w2v_dim, weights=[word2vec_matrix], input_length=N, trainable=False)(main_input)

    # add the second input layer which corresponds to the category embeddings
    subject_input = Input(shape=(N,), dtype='int32', name='subject_input')
    e2 = Embedding(n2v_size, n2v_dim, weights=[node2vec_matrix], input_length=N, trainable=False)(subject_input)

    # add the second input layer which corresponds to the category embeddings
    value_input = Input(shape=(N,), dtype='int32', name='value_input')
    e3 = Embedding(n2v_size, n2v_dim, weights=[node2vec_matrix], input_length=N, trainable=False)(value_input)

    # merge the two embedding layers
    e4 = merge([e, e2, e3], mode='sum')
    drop_out = Dropout(0.3, name='dropout')(e4)

    # pass the input through a BiLSTM model through the previous layer
    bilstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(drop_out)

    # add another drop-out layer
    drop_out = Dropout(0.2)(bilstm)
    flatten = Flatten()(drop_out)
    out = Dense(3, activation='softmax')(flatten)
    
    model = Model(input=[main_input, subject_input, value_input], output=out)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
    return model


'''
    We build a BiLSTM model with attention, which matches two tables based on their column representation.
    The attention mechanism decides which columns have their highest match from the second candidate table
    and then uses this to compute the attention weights which later on are used for classifying the pair
    as either: 'equivalent', 'subPartOf' or 'notAligned'

    The model is based on the paper by Rockta\"schel et al. "Reasoning about Entailment with Neural Attention"
    
    @Credit: This is an adaptation of the implementation of the original paper by https://github.com/shyamupa/snli-entailment
'''


def build_bilstm_col_subject_val_model(w2v_size, w2v_dim, word2vec_matrix, n2v_size, n2v_dim, node2vec_matrix,
                                   lstm_units=100, col_length=10):
    k = 2 * lstm_units
    N = LEN

    # the first layer is the embeddings for the column names
    main_input = Input(shape=(N,), dtype='int32', name='main_input')
    e = Embedding(w2v_size, w2v_dim, weights=[word2vec_matrix], input_length=N, trainable=False)(main_input)

    # add the second input layer which corresponds to the category embeddings
    subject_input = Input(shape=(N,), dtype='int32', name='subject_input')
    e2 = Embedding(n2v_size, n2v_dim, weights=[node2vec_matrix], input_length=N, trainable=False)(subject_input)

    # add the second input layer which corresponds to the category embeddings
    value_input = Input(shape=(N,), dtype='int32', name='value_input')
    e3 = Embedding(n2v_size, n2v_dim, weights=[node2vec_matrix], input_length=N, trainable=False)(value_input)

    # merge the two embedding layers
    e4 = merge([e, e2, e3], mode='sum')
    
    drop_out = Dropout(0.3, name='dropout')(e4)
    
    # pass the input through a BiLSTM model through the previous layer
    bilstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(drop_out)

    # add another drop-out layer
    drop_out = Dropout(0.1)(bilstm)

    # here we add a custom layer which connects the last input of the first table as the input of the second table
    t2 = Lambda(get_H_n, output_shape=(k,), name='h_n')(drop_out)

    # add the layer which encodes the output labels
    Y = Lambda(get_Y, arguments={'xmaxlen': col_length}, name='Y', output_shape=(col_length, k))(drop_out)
    # add the layer which encodes the weight parameter from the LSTMs cells and the hidden states
    Whn = Dense(k, W_regularizer=l2(0.01), name="Wh_n")(t2)
    Whn_x_e = RepeatVector(col_length, name="Wh_n_x_e")(Whn)

    # add the attention layer
    WY = TimeDistributed(Dense(2 * lstm_units, W_regularizer=l2(0.01)), name="WY")(Y)
    merged = merge([Whn_x_e, WY], name='merged', mode='sum')
    M = Activation('tanh', name='M')(merged)

    alpha_ = TimeDistributed(Dense(1, activation='linear'), name="alpha_")(M)
    flat_alpha = Flatten(name="flat_alpha")(alpha_)
    alpha = Dense(col_length, activation='softmax', name="alpha")(flat_alpha)

    Y_trans = Permute((2, 1), name="y_trans")(Y)  # of shape (None,300,20)
    r_ = merge([Y_trans, alpha], output_shape=(k, 1), name="r_", mode=get_R)
    r = Reshape((k,), name="r")(r_)

    Wr = Dense(k, W_regularizer=l2(0.01))(r)
    Wh = Dense(k, W_regularizer=l2(0.01))(t2)
    merged = merge([Wr, Wh], mode='sum')
    h_star = Activation('tanh')(merged)

    out = Dense(3, activation='softmax')(h_star)
    output = out

    model = Model(input=[main_input, subject_input, value_input], output=output)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
    return model


'''
    We build a BiLSTM model with attention, which matches two tables based on their column representation.
    The attention mechanism decides which columns have their highest match from the second candidate table
    and then uses this to compute the attention weights which later on are used for classifying the pair
    as either: 'equivalent', 'subPartOf' or 'notAligned'

    The model is based on the paper by Rockta\"schel et al. "Reasoning about Entailment with Neural Attention"
    
        @Credit: This is an adaptation of the implementation of the original paper by https://github.com/shyamupa/snli-entailment
'''


def build_bilstm_col_subject_model(w2v_size, w2v_dim, word2vec_matrix, n2v_size, n2v_dim, node2vec_matrix,
                                   lstm_units=100, col_length=10):
    k = 2 * lstm_units
    N = col_length

    # the first layer is the embeddings for the column names
    main_input = Input(shape=(N,), dtype='int32', name='main_input')
    e = Embedding(w2v_size, w2v_dim, weights=[word2vec_matrix], input_length=N, trainable=False)(main_input)

    # add the second input layer which corresponds to the category embeddings
    subject_input = Input(shape=(N,), dtype='int32', name='subject_input')
    e2 = Embedding(n2v_size, n2v_dim, weights=[node2vec_matrix], input_length=N, trainable=False)(
        subject_input)

    # merge the two embedding layers
    e3 = merge([e, e2], mode='sum')
    
    drop_out = Dropout(0.3, name='dropout')(e3)
    
    # pass the input through a BiLSTM model through the previous layer
    bilstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(drop_out)

    # add another drop-out layer
    drop_out = Dropout(0.1)(bilstm)

    # here we add a custom layer which connects the last input of the first table as the input of the second table
    t2 = Lambda(get_H_n, output_shape=(k,), name='h_n')(drop_out)

    # add the layer which encodes the output labels
    Y = Lambda(get_Y, arguments={'xmaxlen': col_length}, name='Y', output_shape=(col_length, k))(drop_out)
    # add the layer which encodes the weight parameter from the LSTMs cells and the hidden states
    Whn = Dense(k, W_regularizer=l2(0.01), name="Wh_n")(t2)
    Whn_x_e = RepeatVector(col_length, name="Wh_n_x_e")(Whn)

    # add the attention layer
    WY = TimeDistributed(Dense(2 * lstm_units, W_regularizer=l2(0.01)), name="WY")(Y)
    merged = merge([Whn_x_e, WY], name='merged', mode='sum')
    M = Activation('tanh', name='M')(merged)

    alpha_ = TimeDistributed(Dense(1, activation='linear'), name="alpha_")(M)
    flat_alpha = Flatten(name="flat_alpha")(alpha_)
    alpha = Dense(col_length, activation='softmax', name="alpha")(flat_alpha)

    Y_trans = Permute((2, 1), name="y_trans")(Y)  # of shape (None,300,20)
    r_ = merge([Y_trans, alpha], output_shape=(k, 1), name="r_", mode=get_R)
    r = Reshape((k,), name="r")(r_)

    Wr = Dense(k, W_regularizer=l2(0.01))(r)
    Wh = Dense(k, W_regularizer=l2(0.01))(t2)
    merged = merge([Wr, Wh], mode='sum')
    h_star = Activation('tanh')(merged)

    out = Dense(3, activation='softmax')(h_star)
    output = out

    model = Model(input=[main_input, subject_input], output=output)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
    return model


'''
    We build a BiLSTM model with attention, which matches two tables based on their column representation.
    The attention mechanism decides which columns have their highest match from the second candidate table
    and then uses this to compute the attention weights which later on are used for classifying the pair
    as either: 'equivalent', 'subPartOf' or 'notAligned'
    
    The model is based on the paper by Rockta\"schel et al. "Reasoning about Entailment with Neural Attention"
    
    @Credit: This is an adaptation of the implementation of the original paper by https://github.com/shyamupa/snli-entailment
'''


def build_bilstm_col_model(vocab_size, w2v_dim, embedding_matrix, lstm_units=100, col_length=20):
    k = 2 * lstm_units
    N = col_length
    # the first layer is the embeddings for the column names
    main_input = Input(shape=(N,), dtype='int32', name='main_input')
    e = Embedding(vocab_size, w2v_dim, weights=[embedding_matrix], input_length=N, trainable=False)(main_input)
    drop_out = Dropout(0.1, name='droput')(e)

    # pass the input through a BiLSTM model through the previous layer
    bilstm = Bidirectional(LSTM(lstm_units, return_sequences=True))(drop_out)

    # add another drop-out layer
    drop_out = Dropout(0.1)(bilstm)

    # here we add a custom layer which connects the last input of the first table as the input of the second table
    t2 = Lambda(get_H_n, output_shape=(k,), name='h_n')(drop_out)

    # add the layer which encodes the output labels
    Y = Lambda(get_Y, arguments={'xmaxlen': col_length}, name='Y', output_shape=(col_length, k))(drop_out)
    # add the layer which encodes the weight parameter from the LSTMs cells and the hidden states
    Whn = Dense(2 * lstm_units, W_regularizer=l2(0.01), name="Wh_n")(t2)
    Whn_x_e = RepeatVector(col_length, name="Wh_n_x_e")(Whn)

    # add the attention layer
    WY = TimeDistributed(Dense(2 * lstm_units, W_regularizer=l2(0.01)), name="WY")(Y)
    merged = merge([Whn_x_e, WY], name='merged', mode='sum')
    M = Activation('sigmoid', name='M')(merged)

    alpha_ = TimeDistributed(Dense(1, activation='linear'), name="alpha_")(M)
    flat_alpha = Flatten(name="flat_alpha")(alpha_)
    alpha = Dense(col_length, activation='softmax', name="alpha")(flat_alpha)

    Y_trans = Permute((2, 1), name="y_trans")(Y)  # of shape (None,300,20)
    r_ = merge([Y_trans, alpha], output_shape=(2 * lstm_units, 1), name="r_", mode=get_R)
    r = Reshape((k,), name="r")(r_)

    Wr = Dense(k, W_regularizer=l2(0.01))(r)
    Wh = Dense(k, W_regularizer=l2(0.01))(t2)
    merged = merge([Wr, Wh], mode='sum')
    h_star = Activation('sigmoid')(merged)

    out = Dense(3, activation='softmax')(h_star)
    output = out

    model = Model(input=[main_input], output=output)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['acc'])
    return model

In [6]:
# we store the pre-loaded embeddings so that we do not load that every time we do changes in the class.
word2vec = None
node2vec = None
tables = dict()
entity_cats = dict()
cat_tax = dict()
cat_level = dict()

In [7]:
emb_w2v = None
emb_n2v = None

In [8]:
verse_emb = None

In [55]:

from nltk.corpus import stopwords
from gensim.models import KeyedVectors
import numpy as np
import gzip
import json
import pandas as pd
from pandas import DataFrame

from sklearn.metrics import confusion_matrix
import random
from table import WikiTable
import re, os, sys


class DataLoader:
    def __init__(self):
        # the stop words, which we will use to skip stop words from the column titles
        self.stops = set(stopwords.words('english'))
        self.word2vec_path = ''
        self.node2vec_path = ''
        self.cat_taxonomy_path = ''
        self.entity_category_path = ''
        self.table_data_path = ''
        self.table_data_labels = ''
        self.out_dir = ''

        # data structures that which will hold the necessary data for training the models
        self.entity_cats = dict()
        self.word2vec = None
        self.node2vec = None
        self.vocab_n2v = dict()
        self.vocab_w2v = dict()
        self.DELIM = 0
        self.table_pairs = []
        self.tables = dict()
        self.table_rep = dict()
        self.eval_pairs = []
        self.cat_tax = dict()
        self.cat_parents = dict()
        self.cat_level = dict()


    '''
        Load the word2vec and the node2vec embeddings.
    '''

    def load_embeddings(self):
        self.word2vec = KeyedVectors.load_word2vec_format(self.word2vec_path, binary=False)
        self.node2vec = KeyedVectors.load_word2vec_format(self.node2vec_path, binary=False)

    def construct_vocab(self):
        self.vocab_w2v = {key: idx + 2 for idx, key in enumerate(self.word2vec.vocab)}

        # keep zero for the unknown words
        self.vocab_w2v['UNK'] = 0
        self.vocab_w2v['COL_TOKEN_SPLIT'] = 1

        self.vocab_n2v = {key: idx + 2 for idx, key in enumerate(self.node2vec.vocab)}
        self.vocab_n2v['UNK'] = 0
        self.vocab_n2v['COL_TOKEN_SPLIT'] = 1

        # the delimiter which we use to stitch tables together
        self.DELIM_N2V = [len(self.vocab_n2v)]
        self.DELIM_W2V = [len(self.vocab_w2v)]

    '''
        Load the evaluation dataset
    '''

    def load_evaluation_data(self):
        for table_pair in self.table_pairs:
            table_pair_dict = dict()
            table_pair_dict['col_name'] = []
            table_pair_dict['col_values'] = []
            table_pair_dict['col_subject'] = []
            
            table_a = self.table_rep[table_pair[0]]
            table_b = self.table_rep[table_pair[1]]

            # generate the different representations
            label = table_pair[2]
            
            # concatenate the features for the different tables
            ep_col_name = np.concatenate((np.concatenate(table_a['col_name'].values()), self.DELIM_W2V, np.concatenate(table_b['col_name'].values())), axis=0)
            ep_col_values = np.concatenate((np.concatenate(table_a['col_values'].values()), self.DELIM_N2V, np.concatenate(table_b['col_values'].values())), axis=0)
            ep_col_subject = np.concatenate((np.concatenate(table_a['col_subject'].values()), self.DELIM_N2V, np.concatenate(table_b['col_subject'].values())), axis=0)

            # ep_col_name = np.concatenate((np.concatenate(table_a['col_name'].values(), axis=0), self.DELIM_W2V, np.concatenate(table_b['col_name'].values(), axis=0)), axis=0)
            # ep_col_values = np.concatenate((np.concatenate(table_a['col_values'].values(), axis=0), self.DELIM_N2V, np.concatenate(table_b['col_values'].values(), axis=0)), axis=0)
            # ep_col_subject = np.concatenate((np.concatenate(table_a['col_subject'].values(), axis=0), self.DELIM_N2V, np.concatenate(table_b['col_subject'].values(), axis=0)), axis=0)

            table_pair_dict['col_name'].append(ep_col_name)
            table_pair_dict['col_values'].append(ep_col_values)
            table_pair_dict['col_subject'].append(ep_col_subject)

            table_pair_dict['table_a'] = table_pair[0]
            table_pair_dict['table_b'] = table_pair[1]
            table_pair_dict['label'] = label

            self.eval_pairs.append(table_pair_dict)

    '''
        Load the table pairs for alignment with their table data values and their corresponding label
    '''

    def load_alignment_pairs(self):
        # load the table data only for these entities and the table alignment pairs
        self.table_pairs = []
        
        eval_data = DataFrame.from_csv(self.table_data_labels, sep='\t', index_col=None)
        for row_idx, row in eval_data.iterrows():
            # the label for the table alignment pair
            label = row['label']
            # add the entities in the entity index

            # add the table pairs as a tuple with the label
            tbl_id_a = int(row['tbl_id_a'])
            tbl_id_b = int(row['tbl_id_b'])
            self.table_pairs.append((tbl_id_a, tbl_id_b, label))
            
        # load the table data
        self.load_tables()

    '''
        Load the table data for a set of entities of interest.
    '''

    def load_tables(self):
        # read the table data and take only the tables for the entities in the entity index.
        fin = gzip.open(self.table_data_path, 'rt')

        # return the tables as a dict with the table id as an index.
        self.tables = dict()

        for line in fin:
            if len(line.strip()) == 0:
                continue
            tbl_json = json.loads(line)
            entity = tbl_json['entity']
            sections = tbl_json['sections']
            for section in sections:
                tables_json = section['tables']

                for table in tables_json:
                    tbl = WikiTable()
                    tbl.load_json(json.dumps(table), entity, section, int(table['id']), col_meta_parse=True)
                    tbl.markup = table['markup']
                    self.tables[tbl.table_id] = tbl

    '''
        Constructs the evaluation data which we will use to train our DL model. 
        In this case we will represent the data in the following ways:
        
            1)  The simplest form of the data representation is in terms of the column names.
            2)  We augment the representation with the entities or values present in a column, 
                in case the values do not link to entities or are not textual, 
                then we will represent the column data with a UNK vector.
            3)  Finally, in the case where (2) reflects entities, we will additionally represent
                the data with the corresponding column label (i.e., the LCA category of entities)
                
    '''

    def construct_eval_data(self):
        self.table_rep.clear()
        # generate the appropriate table representation that we can use for the deep learning models.
        for table_id in self.tables:
            table = self.tables[table_id]
            sub_table_rep = dict()

            # generate the column representation, for the columns for which we do not have a word we assign UNK
            col_names_rep = dict()
            col_val_rep = dict()
            col_subj_rep = dict()

            for col_idx, column in enumerate(table.column_meta_data):
                col_subj_rep[col_idx] = []
                col_val_rep[col_idx] = []
                col_names_rep[col_idx] = []
                
                # generate the word embedding for the column name
                col_names_rep[col_idx] = self.column_title_to_idx(column, self.vocab_w2v)

                # generate the column representation based on the entities
                col_values = table.column_meta_data[column]

                # take the subset of values which exist in our entity-category index
                sub_vals = []
                for val in col_values:
                    val_label = re.sub(' ', '_', val)
                    if val_label in self.vocab_n2v:
                        sub_vals.append(val_label)
                    
                for val in sub_vals:
                    col_val_rep[col_idx].append(self.vocab_n2v[val])
                col_val_rep[col_idx].append(self.vocab_n2v['COL_TOKEN_SPLIT'])

                # get the lca category for the values in this column (in case they represent entities)
                if len(col_values) != 0:
                    lca_cats = self.find_lca_category(col_values)

                    if lca_cats is not None:
                        for cat in lca_cats: 
                            cat = 'Category:' + re.sub(' ', '_', cat)
                            if cat not in self.vocab_n2v:
                                continue
                            col_subj_rep[col_idx].append(self.vocab_n2v[cat])
                    else:
                        col_subj_rep[col_idx].append(0)
                
                #distinguish between the column representations
                col_subj_rep[col_idx].append(self.vocab_n2v['COL_TOKEN_SPLIT'])
                col_names_rep[col_idx].append(self.vocab_w2v['COL_TOKEN_SPLIT'])
                col_val_rep[col_idx].append(self.vocab_n2v['COL_TOKEN_SPLIT'])
                
            sub_table_rep['col_name'] = col_names_rep
            sub_table_rep['col_values'] = col_val_rep
            sub_table_rep['col_subject'] = col_subj_rep
            self.table_rep[table_id] = sub_table_rep

    '''
        For a given set of seed entities return their common LCA category. This is in a way representing the subject 
        or class of the given entities.
    '''

    def find_lca_category(self, entities):
        for entity in entities:
            if entity not in self.entity_cats:
                continue
            if entity not in self.cat_parents:
                self.cat_parents[entity] = []
            #load all the categories and the parent categories for an entity
            for cat in self.entity_cats[entity]:
                self.load_cat_parents(cat, self.cat_parents[entity])

        # find the common categories
        entity = entities[0]
        common_cats = set()
        index = 0
        for entity in entities:
            if entity not in self.cat_parents:
                continue
            if index == 0:
                common_cats = set(self.cat_parents[entity])
                index += 1
            else:
                common_cats.intersection(self.cat_parents[entity])

        # get the lowest matching category
        if len(common_cats) != 0:
            common_cat_level = [self.cat_level[cat] for cat in common_cats if cat in self.cat_level]
            if len(common_cat_level) != 0:
                max_level = max(common_cat_level)
                return [cat for cat in common_cats if self.cat_level[cat] == max_level]

        return None

    '''
        Since a column name might have different words, we split and aggregate the word vectors from the resulting words.
    '''

    def column_title_to_idx(self, column, vocab):
        col_words = column.lower().split()
        col_rep = []

        if len(col_words) == 1:
            if col_words[0] in vocab and col_words[0] not in self.stops:
                col_rep.append(vocab[col_words[0]])
            else:
                col_rep.append(vocab['UNK'])
        else:
            for col in col_words:
                if col in vocab and col not in self.stops:
                    col_rep.append(vocab[col])
                else:
                    col_rep.append(vocab['UNK'])
        col_rep.append(vocab['COL_TOKEN_SPLIT'])
        return col_rep

    '''
        Load the parents of a category up to the root.
    '''

    def load_cat_parents(self, cat, parents):
        if cat in self.cat_tax and cat not in parents:
            sub_parents = self.cat_tax[cat]
            parents.append(cat)
            
            for parent in sub_parents:
                self.load_cat_parents(parent, parents)

    '''
        Load the category taxonomy where each node has contains its parents
    '''

    def load_flat_cat_tax(self):
        for line in gzip.open(self.cat_taxonomy_path, 'rt'):
            data = line.strip().split('\t')
            
            parent_cat = data[0]
            child_cat = data[2]
        
            if parent_cat not in self.cat_level:
                self.cat_level[parent_cat] = int(data[1])
            if child_cat not in self.cat_level:
                self.cat_level[child_cat] = int(data[3])

            if child_cat not in self.cat_tax:
                self.cat_tax[child_cat] = []
            self.cat_tax[child_cat].append(parent_cat)

    '''
        Loads the entity categories.
    '''

    def load_entity_cats(self):
        for line in gzip.open(self.entity_category_path, 'rt'):
            data = line.strip().split('\t')
            if len(data) != 2:
                continue

            if data[0] not in self.entity_cats:
                self.entity_cats[data[0]] = []
            self.entity_cats[data[0]].append(data[1])

    '''
        The word2vec embedding matrix.
    '''

    def get_word2vec_matrix(self, emb_dim):
        # This will be the embedding matrix
        embeddings = 1 * np.random.randn(len(self.vocab_w2v) + 2, emb_dim)
        embeddings[0] = 0  # So that the padding will be ignored

        # Build the embedding matrix
        for word, index in self.vocab_w2v.items():
            if word in self.word2vec.vocab:
                embeddings[index] = self.word2vec.word_vec(word)[:emb_dim]

        # del self.word2vec
        return embeddings

    '''
           The node2vec embedding matrix.
    '''

    def get_node2vec_matrix(self, emb_dim):
        # This will be the embedding matrix
        embeddings = 1 * np.random.randn(len(self.vocab_n2v) + 2, emb_dim)
        embeddings[0] = 0  # So that the padding will be ignored

        # Build the embedding matrix
        for word, index in self.vocab_n2v.items():
            if word in self.node2vec.vocab:
                embeddings[index] = self.node2vec.word_vec(word)[:emb_dim]
        # del self.node2vec
        return embeddings


## Load the data and train the models.

In [56]:
base_dir = '/home/fetahu/wiki_tables/data/'

In [57]:
# create the class DataLoader
loader = DataLoader()

In [79]:
loader.cat_taxonomy_path = base_dir + 'category_data/flat_cat_taxonomy.tsv.gz'
loader.table_data_path = '../data/table_data/html_data/structured_html_table_data_ground_truth.json.gz'
loader.word2vec_path = base_dir + 'embeddings/glove.6B.300d.emb.gz'
loader.node2vec_path = base_dir + 'embeddings/category_entity_label_node2vec.emb.gz'
loader.entity_category_path = base_dir + 'category_data/article_cats_201708.tsv.gz'
loader.table_data_labels = '../data/gt_data/table_pair_evaluation_eq_sub_irrel_labels.tsv'
loader.out_dir = base_dir

In [80]:
# load first the embeddings
word2vec = None
node2vec = None
if word2vec is None and node2vec is None:
    loader.load_embeddings()
    word2vec = loader.word2vec
    node2vec = loader.node2vec
else:
    loader.word2vec = word2vec
    loader.node2vec = node2vec
'Loaded word embeddings with %d entries and node2vec embeddings with %d entries' % (len(loader.word2vec.vocab), len(loader.node2vec.vocab))

'Loaded word embeddings with 400000 entries and node2vec embeddings with 3374828 entries'

In [81]:
# construct the vocabularies
loader.construct_vocab()
'Loaded the word and entity vocabularies'

'Loaded the word and entity vocabularies'

In [82]:
len(entity_cats)

5882458

In [62]:
# entity_cats = None
if entity_cats is None or len(entity_cats) == 0:
    loader.load_entity_cats()
    entity_cats = loader.entity_cats
else:
    loader.entity_cats = entity_cats
'Loaded the entity categories for %d entities' % (len(loader.entity_cats))

'Loaded the entity categories for 5882458 entities'

In [63]:
if cat_tax is None or len(cat_tax) == 0:
    loader.load_flat_cat_tax()
    cat_tax = loader.cat_tax
    cat_level = loader.cat_level
else:
    loader.cat_tax = cat_tax
    loader.cat_level = cat_level
'Loaded the category taxonomy with %d entries' % (len(loader.cat_tax))

'Loaded the category taxonomy with 1324003 entries'

In [103]:
tables = None
if tables is not None:
    loader.tables = tables
    loader.load_alignment_pairs()
else:
    loader.load_alignment_pairs()
    tables = loader.tables
'Loaded the table data with %d entries' % (len(loader.tables))

'Loaded the table data with 18369 entries'

In [104]:
%%time
loader.construct_eval_data()

In [105]:
loader.load_evaluation_data()
'Constructed the evaluation data for all tables'

'Constructed the evaluation data for all tables'

In [85]:
# create the matrices for the embeddings
emb_w2v = loader.get_word2vec_matrix(256)
emb_n2v = loader.get_node2vec_matrix(256)

In [106]:
X = []
X_subj = []
X_val = []
Y = []

for inst in loader.eval_pairs:
    X += inst['col_name']
    X_subj += inst['col_subject']
    X_val += inst['col_values']
    Y += [inst['label']]

In [107]:
encoder = LabelBinarizer()
Y_fit = encoder.fit_transform(Y)

In [108]:
encoder.classes_

array(['equivalent', 'noalignment', 'subpartof'], dtype='|S11')

In [109]:
LEN = max(set([len(x) for x in X]) | set([len(x) for x in X_val]) | set([len(x) for x in X_subj]))

In [110]:
X_val = pad_sequences(X, maxlen=LEN,value=loader.vocab_w2v['UNK'], padding='pre')
X_val_subj = pad_sequences(X_subj, maxlen=LEN,value=loader.vocab_n2v['UNK'], padding='pre')
X_val_val = pad_sequences(X_val, maxlen=LEN,value=loader.vocab_n2v['UNK'], padding='pre')

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X_val, Y_fit, test_size=0.3, random_state=1)
X_train_subj, X_test_subj, y_train_subj, y_test_subj = train_test_split(X_val_subj, Y_fit, test_size=0.3, random_state=1)
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_val_val, Y_fit, test_size=0.3, random_state=1)

# TableNet Model -- Column Title Word Representation

In [ ]:
%%time
tablenet_desc = build_bilstm_col_model(len(emb_w2v), 256, emb_w2v, 50, LEN)
tablenet_desc.fit(x=X_train, y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [421]:
y_pred_tablenet_val = tablenet_desc.predict([X_test])

In [422]:
mat_tablenet_val = cm(y_test.argmax(axis=1), y_pred_tablenet_val.argmax(axis=1))

In [424]:
print evaluation_metrics_report(mat_tablenet_val, encoder.classes_, 'TableNet - Column', 30)

# TableNet Model - Column LCA representation

In [ ]:
tablenet_desc_lca = build_bilstm_col_subject_model(len(emb_w2v), 256, emb_w2v, len(emb_n2v), 256, emb_n2v, 50, LEN)

In [ ]:
tablenet_desc_lca.fit(x=[X_train, X_train_subj], y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [416]:
y_pred_tablenet_desc_lca = tablenet_desc_lca.predict([X_test, X_test_subj])

In [417]:
mat_tablenet_desc_lca = cm(y_test.argmax(axis=1), y_pred_tablenet_desc_lca.argmax(axis=1))

In [419]:
print evaluation_metrics_report(mat_tablenet_desc_lca, encoder.classes_, 'TableNet - Column+LCA', 30)

# TableNet Model - Column title, LCA and Value representation

In [ ]:
tablenet_desc_val_lca = build_bilstm_col_subject_val_model(len(emb_w2v), 256, emb_w2v, len(emb_n2v), 256, emb_n2v, 50, LEN)

In [ ]:
tablenet_desc_val_lca.fit(x=[X_train, X_train_subj, X_train_val], y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [119]:
y_pred_tablenet_col_lca_val = tablenet_desc_val_lca.predict([X_test, X_test_subj, X_test_val])

In [120]:
m_tablenet_col_lca_val = cm(y_test.argmax(axis=1), y_pred_tablenet_col_lca_val.argmax(axis=1))

In [121]:
print evaluation_metrics_report(m_tablenet_col_lca_val, encoder.classes_, 'TableNet - Column+VAL+LCA', 50)

# TableNet Model - Column title, and value representation

In [518]:
tablenet_desc_val = build_bilstm_col_subject_model(len(emb_w2v), 256, emb_w2v, len(emb_n2v), 256, emb_n2v, 50, LEN)

In [ ]:
tablenet_desc_val.fit(x=[X_train, X_train_val], y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [521]:
y_pred_tablenet_col_val = tablenet_desc_val.predict([X_test, X_test_val])
m4 = cm(y_test.argmax(axis=1), y_pred_tablenet_col_val.argmax(axis=1))

In [522]:
print evaluation_metrics_report(m4, encoder.classes_, 'TableNet - Column+VAL', 30)

# BiLSTM baseline - column title, LCA, value representation

In [ ]:
bilstm_baseline_lca = build_bilstm_baseline_w2v_lca_val(len(emb_w2v), 256, emb_w2v, len(emb_n2v), 256, emb_n2v, 50, LEN)

In [ ]:
bilstm_baseline_lca.fit(x=[X_train, X_train_subj, X_train_val], y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [115]:
y_pred_blst_lca_val = bilstm_baseline_lca.predict([X_test, X_test_subj, X_test_val])
blstm_lca_scores = cm(y_test.argmax(axis=1), y_pred_blst_lca_val.argmax(axis=1))

In [116]:
print evaluation_metrics_report(blstm_lca_scores, encoder.classes_, 'BiLSTM - Column+VAL+LCA', 10)

# BiLSTM baseline - column title, VAL representation

In [505]:
bilstm_baseline_w2v_val = build_bilstm_baseline_w2v_lca(len(emb_w2v), 256, emb_w2v, len(emb_n2v), 256, emb_n2v, 50, LEN)

In [506]:
bilstm_baseline_w2v_val.fit(x=[X_train, X_train_val], y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [510]:
y_pred_b_w2v_val = bilstm_baseline_w2v_val.predict([X_test, X_test_subj])
b_w2v_val_mat = cm(y_test.argmax(axis=1), y_pred_b_w2v_val.argmax(axis=1))

In [511]:
print evaluation_metrics_report(b_w2v_val_mat, encoder.classes_, 'BiLSTM - Column+VAL', 50)

# BiLSTM baseline - column title, LCA representation

In [448]:
bilstm_baseline_w2v_lca = build_bilstm_baseline_w2v_lca(len(emb_w2v), 256, emb_w2v, len(emb_n2v), 256, emb_n2v, 50, LEN)

In [449]:
bilstm_baseline_w2v_lca.fit(x=[X_train, X_train_subj], y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [451]:
y_pred_b_w2v_lca = bilstm_baseline_w2v_lca.predict([X_test, X_test_subj])
b_w2v_lca_mat = cm(y_test.argmax(axis=1), y_pred_b_w2v_lca.argmax(axis=1))

In [452]:
print evaluation_metrics_report(b_w2v_lca_mat, encoder.classes_, 'BiLSTM - Column+LCA', 10)

# BiLSTM baseline - column title representation

In [496]:
bilstm_baseline_w2v = build_bilstm_baseline_w2v(len(emb_w2v), 256, emb_w2v, 50, LEN)

In [ ]:
bilstm_baseline_w2v.fit(x=X_train, y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [500]:
y_pred_b_w2v = bilstm_baseline_w2v.predict(X_test)

In [501]:
b_w2v_mat = cm(y_test.argmax(axis=1), y_pred_b_w2v.argmax(axis=1))

In [502]:
print evaluation_metrics_report(b_w2v_mat, encoder.classes_, 'BiLSTM - Column', 50)

# LSTM baseline - column title, and VAL representation

In [470]:
model_lstm_w2v_val = build_lstm_baseline_w2v_lca(len(emb_w2v), 256, emb_w2v, len(emb_n2v), 256, emb_n2v, 50, LEN)

In [487]:
model_lstm_w2v_val.fit(x=[X_train, X_train_val], y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [507]:
y_pred_lstm_w2v_val = model_lstm_w2v_val.predict([X_test, X_test_subj])
lstm_w2v_val_mat = cm(y_test.argmax(axis=1), y_pred_lstm_w2v_val.argmax(axis=1))

In [508]:
print evaluation_metrics_report(lstm_w2v_val_mat, encoder.classes_, 'LSTM - Column + VAL', 30)

# LSTM baseline - column title, and LCA representation

In [ ]:
model_lstm_w2v = build_lstm_baseline_w2v_lca(len(emb_w2v), 256, emb_w2v, len(emb_n2v), 256, emb_n2v, 50, LEN)
model_lstm_w2v.fit(x=[X_train, X_train_subj], y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [463]:
y_pred_lstm_w2v_lca = model_lstm_w2v.predict([X_test, X_test_subj])
lstm_w2v_lca_mat = cm(y_test.argmax(axis=1), y_pred_lstm_w2v_lca.argmax(axis=1))

In [464]:
print evaluation_metrics_report(lstm_w2v_lca_mat, encoder.classes_, 'LSTM - Column + LCA', 10)

# LSTM baseline  - column title representation

In [ ]:
model_lstm_w2v_col = build_lstm_baseline_w2v(len(emb_w2v), 256, emb_w2v, 50, LEN)

In [490]:
model_lstm_w2v_col.fit(x=X_train, y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [491]:
y_pred_lstm_w2v_col = model_lstm_w2v_col.predict(X_test)
lstm_w2v_col_mat = cm(y_test.argmax(axis=1), y_pred_lstm_w2v_col.argmax(axis=1))

In [492]:
print evaluation_metrics_report(lstm_w2v_col_mat, encoder.classes_, 'LSTM - Column', 50)

# LSTM baseline - column title, LCA, value representation

In [ ]:
model_lstm_w2v_lca_val = build_lstm_baseline_w2v_lca_val(len(emb_w2v), 256, emb_w2v, len(emb_n2v), 256, emb_n2v, 50, LEN)

In [ ]:
model_lstm_w2v_lca_val.fit(x=[X_train, X_train_subj, X_train_val], y=y_train, batch_size=100, epochs=50, validation_split=0.1)

In [494]:
y_pred_lstm_w2v_col_lca_val = model_lstm_w2v_lca_val.predict([X_test, X_test_subj, X_test_val])
lstm_w2v_col_lca_val_mat = cm(y_test.argmax(axis=1), y_pred_lstm_w2v_col_lca_val.argmax(axis=1))

In [495]:
print evaluation_metrics_report(lstm_w2v_col_lca_val_mat, encoder.classes_, 'LSTM - Column+VAL+LCA', 10)